In [1]:
import pandas as pd
import tqdm
import re

from konlpy.corpus import kolaw
from konlpy.tag import Okt, Hannanum, Kkma, Komoran, Twitter, Mecab
# import MeCab
from konlpy.utils import concordance, pprint

import matplotlib.pyplot as plt
import seaborn as sns

# 한글깨짐 설정
import matplotlib.font_manager as fm

# print(fm.findSystemFonts(fontpaths=None, fontext='ttf'))
# font_location = '/usr/share/fonts/truetype/NanumGothic.ttf'  #font 경로 설정
# font_name = fm.FontProperties(fname=font_location).get_name()
# plt.rc('font', family=font_name)
# plt.rc('font', size=12)        # 기본 폰트 크기

import warnings
warnings.filterwarnings('ignore')

import gensim
gensim.__version__
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus
from gensim.models.wrappers import LdaMallet


In [2]:
df = pd.read_csv('../Data/novel.csv', sep='\t')
# df = pd.read_csv('./Data/novel_total_final.csv', on_bad_lines='skip')
df

,id,itemId,title,pubDate,coverSmallUrl,coverLargeUrl,publisher,priceStandard,customerReviewRank,author,isbn,link,story,review,piece
0,1,349572433,주술회전 1: 떠나가는 여름과 돌아오는 가을,20210525,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,서울미디어코믹스(서울문화사),7800,0.0,Gege Akutami(아쿠타미 제제),9791136734518,http://book.interpark.com/blog/integration/pro...,"대인기 다크 액션 판타지, 「주술회전」소설 1권 발행!!이타도리와 후시구로의 아키하...",NaN,NaN
1,2,348109712,흐르는 강물처럼(40주년 기념판),20210415,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,연암서가,15000,0.0,노먼 매클린,9791160870763,http://book.interpark.com/blog/integration/pro...,영화 〈흐르는 강물처럼〉의 원작소설영화보다 더한 감동이 굽이쳐 흐른다! 〈흐르는 강...,NaN,나는 1981년 몬태나 주를 방문하던 중에 내 친구 토머스 맥구언과 함께 미국 서부...
2,3,340692027,백작가의 비밀스런 시녀님 세트,20201027,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,필프리미엄에디션,38000,8.0,백주아,9791165655778,http://book.interpark.com/blog/integration/pro...,백주아 장편소설 『백작가의 비밀스런 시녀님 세트』. 가난한 소작농의 딸 폴라. 우연...,〈1권〉가난한 소작농의 딸 폴라.우연한 계기로 명망 높은 벨루니타 백작가의 사용인으...,NaN
3,4,343464716,신세계에서 2,20201125,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,해냄출판사,18000,9.9,기시 유스케(貴志祐介),9788965740322,http://book.interpark.com/blog/integration/pro...,일본 SF대상 수상작 & 서점대상 제6위미래의 가상세계에 빗대어 현 인류의 모순을 ...,"순백의 아이들이 평화롭게 살아가는 머나먼 미래, 그 아름다운 유토피아에 가려진 잔혹...",X. 얼굴 없는 소년. 누구보다 사랑했던 소년. 그런데 지금은 얼굴도 떠올릴 수 없...
4,5,340768143,추리소설가의 살인사건,20201028,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,소미미디어,15800,9.5,히가시노 게이고(東野圭吾),9791166111983,http://book.interpark.com/blog/integration/pro...,"추리 소설가, 편집자, 독자가 등장하는 추리소설,히가시노 게이고만이 쓸 수 있는 색...","이 작품집, 꽤나 웃기다! 한 번 읽기 시작했다면 멈출 수 없다! 올해의 페이지 터...",그것은 하마사키 회계사무소에서 온 서류였다. 소장인 하마사키 고로는 내 고등학교 동...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,1114,345966624,어니언 마말레이드 2,20210121,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,동아,12800,0.0,백서하,9791163024453,http://book.interpark.com/blog/integration/pro...,"사랑에 빠진 위그의 뒤통수를 치고,이디에트 가문의 약점을 손에 쥐게 된 비비안. “...",NaN,“상속법 관련 문제에 대해 의견을 묻고 싶네.”“죄송하지만 그 부분에 대해서는 귀족...
1114,1115,343789806,갑의 품격 1,20201130,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,가연,14500,0.0,김나든,9788968970818,http://book.interpark.com/blog/integration/pro...,"네이버 시리즈 다운로드 수 84만, 실시간 랭킹 1위.혜성처럼 나타난 김나든 작가의...","서울의 끝자락 판자촌 태생, 부모가 누군지도 모르는 고아. 보육원을 떠돌던 예일은 ...","생전 처음 보는 사람 앞에서 그렇게 예일은 펑펑 울었다. 산발이 된 머리도, 그에게..."
1115,1116,342291271,진실에 갇힌 남자,20201109,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,북로드,14800,9.5,데이비드 발다치,9791158791490,http://book.interpark.com/blog/integration/pro...,“진실이 늘 우리를 자유롭게 하는 것은 아니에요.때론 우리를 가두는 감옥이 될 수도...,"죽음을 눈앞에 두고 자신의 무죄를 주장하던 남자, 이미 죽은 것이나 다름없는 그가 ...","데커는 언젠가 땅 밑에 누워 있는 가족에게 자신도 합류하게 될 그날까지, 매년 몰리..."
1116,1117,342332559,검사 김서진 4,20201008,https://bimage.interpark.com/partner/goods_ima...,https://bimage.interpark.com/partner/goods_ima...,로크미디어,8000,0.0,이해날,9791135485893,http://book.interpark.com/blog/integration/pro...,No방심 작가 이해날의 검사물 리턴즈!그 검사의 목숨을 건 외줄 타기가 시작된다! ...,NaN,NaN


In [3]:
# story와 review 데이터 형태소 분리 작업

def wordSep(df):
    story = df['story']
    story = story.fillna('')

    review = df['review']
    review = review.fillna('')

    stopwords = {'하게', '그것', '되', '선보이는', '해서', '빠져', '있으며', '문학상', '소설가', '작품', '하는데', '온',
                '있었다', '로부터', '이제', '가는', '뒤', '하기', '받아', '되지', '도', '있을', '된', '과', '우리', '따라', 
                '하며', '했던', '앞', '받는', '되면서', '통해', '부', '해', '본', '일까', '그러던', '못', '있고', '또', 
                '하면', '없는', '없었던', '발간', '한다', '번', '있게', '해야', '나가는', '이어', '잘', '문장', '이자', 
                '국문학', '있을까', '더욱', '대한', '되는', '미', '관', '준다', '된다', '듯', '점', '중', '권', '네', '와', 
                '낸', '이런', '무엇', '하는', '냈다', '연재', '다섯', '입니다', '받으며', '로', '동안', '갈', '두', '세', 
                '현대문학', '수', '그려', '찾아', '출', '되는데', '모든', '있도록', '했다', '않은', '하면서', '없을', '있지만', 
                '내', '최고', '사는', '이번', '가지', '되고', '한', '속', '개정판', '다른', '될', '가장', '이후', '할', 
                '베스트셀러', '은', '는', '이었던', '선정', '이', '없었다', '알', '어떻게', '담아', '담겨', '안', '단편소설',
                '년', '없고', '담고', '후', '하던', '수상작', '또한', '그', '간', '있는', '만나', '않는다', '게', '위해', 
                '인물', '채', '있다고', '문학', '만들어', '많은', '대해', '감', '이래', '이러한', '싶은', '써', '번역', 
                '전', '시리즈', '않는', '호', '보다', '것', '같은', '독자', '담긴', '전집', '곳', '넘나', '때문', '있다면', 
                '등', '거', '이야기', '그렇다면', '담은', '작가', '소설', '있어', '해온', '넘어', '모두', '이름', '의해', 
                '첫', '같다', '않고', '저', '다룬', '편', '자', '이를', '주는', '장편소설', '지', '드는', '있', '였다', 
                '하나', '하여', '아닐', '김', '되었다', '제', '지금', '데', '데뷔', '있다', '작', '있었던', '아니라', 
                '한편', '위', '둘', '바로', '있다는', '하지', '낸다', '그런', '란', '였던', '다시', '받고', '를', '쓴', 
                '어떤', '하다', '이었다', '되어', '인해', '보여준다', '책', '저자', '신작', '더', '장', '대', '가진', '개', 
                '아니다', '가', '수록', '아닌', '하고', '없다', '않았다', '단편', '로서', '있던', '때', '문제', '의', 
                '하는', '된', '한', '한다', '할', '출', '해', '된다', '는', '하며', '했다', '되는', '하게', '되었다', 
                '되어', '가', '했던', '될', '하여', '그려', '낸', '되고', '하지', '하기', '와', '않는', '하면서', 
                '하다', '잘', '하고', '하는데', '쓴', '않고', '하던', '되는데', '않은', '담은', 
                '사는', '가진', '냈다', '대', '받고', '이었다', '해야', '가는', '해온', 
                '싶은', '받는', '이를', '않는다', '만나', '담아', '되면서', '담고', '다룬', '보여준다', '따라', 
                '보다',  '되지', '주는', '본', '찾아', '써', '넘어', 
                '넘나', '되', '선보이는', '준다', '해서', '일까', '만들어', '이었던', '받아', '받으며', 
                '담긴', '담겨', '갈', '하면', '나가는', '빠져', '낸다', '였다', '였던', '지', '이어', '드는' , '않았다',
                '발간', '문학', '권', '를', '있었다', '가장', '권', '이자', '두', '온', '제',
                '소설가', '김','그런', '곳', '더'}

    necessary_type = ('Noun', 'Verb', 'Adjective')

    sep_story = []

    for s in story:
        okt = Okt()
        story_sep = okt.pos(s)
        sentence = ''
        for word,typ in story_sep:
            if (word not in stopwords) and (typ in necessary_type):
                sentence += word + ' '
        sep_story.append(sentence)

    sep_review = []

    for r in review:
        okt = Okt()
        review_sep = okt.pos(r)
        sentence = ''
        for word,typ in review_sep:
            if (word not in stopwords) and (typ in necessary_type):
                sentence += word + ' '
        sep_review.append(sentence)

    analysis_data = [s+r for s,r in zip(sep_story, sep_review)]

    return analysis_data

In [4]:
def lda_coh(stories, no_below=5, num_topics=4, data_word=None, data_words=None):
    
    # 이야기 전체 형태소 분리
    if data_word == None:
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
        
    
    if data_word == 'stop':
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
            
        return data_words
    
    # id2word, corpus 추출
    id2word = corpora.Dictionary(data_words)
    id2word.filter_extremes(no_below = no_below) # no_below 회 이하로 등장한 단어는 삭제
    texts = data_words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    mallet_path = './mallet-2.0.8/bin/mallet' 
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, random_seed=12, id2word=id2word)

    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    
    return texts, corpus, ldamallet, coherence_ldamallet

def lda_coh_id2(stories, no_below=5, num_topics=4, data_word=None, data_words=None):
    
    # 이야기 전체 형태소 분리
    if data_word == None:
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
        
    
    if data_word == 'stop':
        data_words = []
        for story in stories:
            data = list(str(story).split())
            data_words.append(data)
            
        return data_words
    
    # id2word, corpus 추출
    id2word = corpora.Dictionary(data_words)
    id2word.filter_extremes(no_below = no_below) # no_below 회 이하로 등장한 단어는 삭제
    texts = data_words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    mallet_path = './mallet-2.0.8/bin/mallet' 
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, random_seed=12, id2word=id2word)

    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    
    return texts, corpus, ldamallet, coherence_ldamallet, id2word

def format_topics_sentences(texts, corpus, data_words, ldamallet, Data=df):
    # Init output

    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamallet[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamallet[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamallet.show_topic(topic_num, topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    # print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(data_words)
    # sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df = pd.concat([Data, sent_topics_df], axis=1)

    return sent_topics_df

In [5]:
def lda_make_df(df):
    analysis_data = wordSep(df)
    texts, corpus, ldamallet, coherence_ldamallet, id2word = lda_coh_id2(analysis_data, num_topics=10)
    data_words = lda_coh(analysis_data, num_topics=10, data_word='stop')
    df_topic_sents_keywords = format_topics_sentences(texts, corpus, data_words, ldamallet, Data=df)

    return df_topic_sents_keywords

In [6]:
df_lda = lda_make_df(df)

[0.007s][warning][os,thread] Attempt to protect stack guard pages failed (0x0000000169218000-0x0000000169224000).
[0.007s][warning][os,thread] Attempt to deallocate stack guard pages failed.


Mallet LDA: 10 topics, 4 topic bits, 1111 topic mask
Data loaded.
max tokens: 2650
total tokens: 483659
<10> LL/token: -10.13183
<20> LL/token: -9.6069
<30> LL/token: -9.38559
<40> LL/token: -9.2791

0	5	역사 한국 시대 전쟁 대표 세기 화 임꺽정 당시 교수 일본 문화 철학 고전 현대 중국 소개 영화 판 하였다 
1	5	그녀 남자 인생 죽음 여자 눈 인간 아들 순간 일본 향 욕망 운명 딸 여성 결혼 아내 코 소녀 당신 
2	5	살 아이 세상 학교 꿈 주인공 엄마 생각 부모 모습 시선 여성 학생 상황 질문 새로운 공부 미래 폭력 얼마나 
3	5	사회 시간 길 아버지 현실 시대 세계 여행 생 세대 여성 자기 절망 선택 모습 함 회사 한국 상황 의미 
4	5	한국 박완서 소세키 결정 사진 시대 시절 현재 다양한 글 메 사회 새로운 언어 수학 서점 쓰 등단 마지막 예술 
5	5	드라마 판타지 왕 로맨스 여인 조선 스토리 악마 손 운명 감동 세월 세상 달 제작 꽃 웹소설 주인공 만큼 그녀 
6	5	마을 모습 세상 작은 리 누구 자리 발견 사실 서로 목소리 거리 생각 눈 생활 존재 발 사이 황 씨 
7	5	세계 인간 미국 존재 주인공 인류 미래 편의 우주 도시 새로운 지구 장르 등장 과학 소재 공포 상상 언어 수상 
8	5	사건 진실 미스터리 아이 가족 친구 살인 청소년 스릴러 반전 범인 경찰 마지막 추리 죽음 비밀 편지 전개 형사 발견 
9	5	마음 집 시간 청소년 생각 서로 엄마 소년 순간 날 감정 건 관계 세상 기억 아이 친구 몸 가족 위로 

<50> LL/token: -9.21743
<60> LL/token: -9.18181
<70> LL/token: -9.15192
<80> LL/token: -9.14128
<90> LL/token: -9.12767


In [9]:
df_lda = df_lda[['id', 'Dominant_Topic', 'Topic_Keywords']]
for i,t,k in zip(df_lda['id'], df_lda['Dominant_Topic'], df_lda['Topic_Keywords']):
    print(i, t, k)

1 8 사건, 미스터리, 일본, 진실, 살인, 스릴러, 범인, 반전, 경찰, 추리
2 0 역사, 한국, 시대, 대표, 전쟁, 교수, 임꺽정, 세기, 문화, 화
3 9 시간, 마음, 집, 세상, 서로, 생각, 순간, 감정, 관계, 기억
4 7 세계, 인간, 미국, 인류, 주인공, 프랑스, 세상, 과학, 지구, 우주
5 8 사건, 미스터리, 일본, 진실, 살인, 스릴러, 범인, 반전, 경찰, 추리
6 3 사회, 현실, 세계, 여성, 인간, 새로운, 세상, 존재, 서사, 여행
7 1 그녀, 남자, 죽음, 여자, 인생, 날, 아버지, 아들, 딸, 눈
8 8 사건, 미스터리, 일본, 진실, 살인, 스릴러, 범인, 반전, 경찰, 추리
9 2 아이, 청소년, 친구, 엄마, 학교, 꿈, 주인공, 가족, 고민, 어른
10 7 세계, 인간, 미국, 인류, 주인공, 프랑스, 세상, 과학, 지구, 우주
11 5 드라마, 판타지, 왕, 로맨스, 조선, 악마, 세상, 마음, 세월, 손
12 5 드라마, 판타지, 왕, 로맨스, 조선, 악마, 세상, 마음, 세월, 손
13 9 시간, 마음, 집, 세상, 서로, 생각, 순간, 감정, 관계, 기억
14 4 한국, 박완서, 결정, 사진, 소세키, 시대, 인간, 글, 메, 언어
15 8 사건, 미스터리, 일본, 진실, 살인, 스릴러, 범인, 반전, 경찰, 추리
16 3 사회, 현실, 세계, 여성, 인간, 새로운, 세상, 존재, 서사, 여행
17 5 드라마, 판타지, 왕, 로맨스, 조선, 악마, 세상, 마음, 세월, 손
18 7 세계, 인간, 미국, 인류, 주인공, 프랑스, 세상, 과학, 지구, 우주
19 8 사건, 미스터리, 일본, 진실, 살인, 스릴러, 범인, 반전, 경찰, 추리
20 2 아이, 청소년, 친구, 엄마, 학교, 꿈, 주인공, 가족, 고민, 어른
21 8 사건, 미스터리, 일본, 진실, 살인, 스릴러, 범인, 반전, 경찰, 추리
22 9 시간, 마음, 집, 세상, 서로, 생각, 순간, 감정, 관계, 기억
23 7 세계, 인간, 미국

In [ ]:
df_topic = df_lda[['id', 'Dominant_Topic', 'Topic_Keywords']]

In [ ]:
df_topic.to_csv('novel_topic_tab.csv', header = True, index = False, encoding='utf-8-sig', sep = '\t')

In [ ]:
df_topic = pd.read_csv('novel_topic_tab.csv', sep='\t')


In [ ]:
df_topic.pivot_table('Dominant_Topic', 'Topic_Keywords').reset_index().sort_values('Dominant_Topic')